In [1]:
cd ../

/nfs/home/dgranziol/kfac-curvature


In [7]:
import argparse
import time
import tabulate

import numpy as np
import matplotlib.pyplot as plt

from gpytorch.utils.lanczos import lanczos_tridiag

import torch

from curvature import data, models, losses, utils
from curvature.methods.swag import SWAG
import scipy.io as sio

In [19]:
dataset = 'CIFAR100'
data_path = '/nfs/home/dgranziol/kfac-curvature/data'
#LOGISTIC REGRESSION MNIST
ckpt = '/nfs/home/dgranziol/kfac-curvature/out/CIFAR100/PreResNet164/SSGDMN/seed=1_epoch_freq=1_curvaturesize=32_warmstart=-1_mom=0.0_wd=0.0005_swastart=200.0_numepochs=300/checkpoint-00000.pt'

dataset = ckpt.split('/out/')[1].split('/')[0]
model = ckpt.split('/seed=')[0].split('/')[7]
network = model
epochsave = int(ckpt.split('checkpoint-')[1].split('.')[0])

# #MLP small MNIST
# ckpt = '/nfs/home/dgranziol/kfac-curvature/out/MNIST/MLP_sdp/SGD/seed=1_lr=0.03_mom=0.9_wd=0.0_batchsize=128_numepochs=200/checkpoint-00200.pt'


#model = 'Logistic'
print('Using model %s' % model)
#from curvature.models import logistic_regression
model_cfg = getattr(models,model)

Using model PreResNet164


In [20]:
batch_size = 1024

datasets, num_classes = data.datasets(
    dataset,
    data_path,
    transform_train=model_cfg.transform_test,
    transform_test=model_cfg.transform_test,
    use_validation=False,
    train_subset=None,
    train_subset_seed=None,
)
loader = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
# full_datasets, _ = data.datasets(
#     dataset,
#     data_path,
#     transform_train=model_cfg.transform_train,
#     transform_test=model_cfg.transform_test,
#     use_validation=False,
# )
# full_loader = torch.utils.data.DataLoader(
#     full_datasets['train'],
#     batch_size=batch_size,
#     shuffle=False,
#     num_workers=4,
#     pin_memory=True
# )

Loading CIFAR100 from /nfs/home/dgranziol/kfac-curvature/data
Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified
Using train (50000) + test (10000)


In [21]:
model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs)
print('Loading %s' % ckpt)
checkpoint = torch.load(ckpt)
model.load_state_dict(checkpoint['state_dict'])

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#device = torch.device('cpu')
model.to(device)
num_parametrs = sum([p.numel() for p in model.parameters()])
criterion = losses.cross_entropy


Loading /nfs/home/dgranziol/kfac-curvature/out/CIFAR100/PreResNet164/SSGDMN/seed=1_epoch_freq=1_curvaturesize=32_warmstart=-1_mom=0.0_wd=0.0005_swastart=200.0_numepochs=300/checkpoint-00000.pt


In [22]:
num_parametrs

1726388

In [31]:
list(model.parameters())[0]

Parameter containing:
tensor([[[[-6.5160e-03,  3.3137e-03, -2.3639e-03],
          [ 2.9666e-03,  1.6227e-02,  6.7247e-03],
          [ 6.2152e-03,  1.4916e-02,  6.9862e-03]],

         [[-1.2350e-02, -4.1195e-03, -5.7482e-03],
          [-3.8080e-03,  9.1157e-03,  4.0834e-03],
          [ 3.9779e-03,  1.4800e-02,  1.0351e-02]],

         [[-1.1762e-02, -1.2606e-02, -1.3136e-02],
          [-8.4863e-03, -4.6814e-03, -6.6693e-03],
          [-1.8965e-03,  3.4121e-03,  2.4678e-03]]],


        [[[-1.7663e-02, -2.9104e-03, -1.8182e-03],
          [-8.0463e-04,  2.0938e-02,  1.5816e-02],
          [ 6.0264e-03,  1.7833e-02,  1.5283e-02]],

         [[-2.3648e-02, -1.2174e-02, -4.9106e-03],
          [-1.2123e-02,  1.1065e-02,  1.6255e-02],
          [-9.0387e-04,  1.9321e-02,  2.7023e-02]],

         [[-1.4247e-02, -2.4422e-02, -2.4560e-02],
          [-1.6559e-02, -1.3535e-02, -9.5008e-03],
          [-1.1954e-02, -3.4330e-03,  7.2356e-03]]],


        [[[ 4.3180e-01, -1.0529e+00, -4.6975

In [24]:
zerocounts = []
import time
for k in range(0,10):
    print(k)
    t0 = time.time()
    randomnumber = np.random.randint(1, num_parametrs)
    #getting the full Hessian
    model.zero_grad()
    N = len(loader.dataset)
    #a = torch.zeros(1,num_parametrs).cuda()
    
    for input, target in loader:
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)
        loss, _, _ = criterion(model, input, target)
        loss *= input.size()[0] / N
        #print(loss)
        #loss.backward(retain_graph=True)
        #loss.backward()
        grad_list = torch.autograd.grad(loss, model.parameters(), create_graph=True)
        grad_i = torch.cat([g.view(-1) for g in grad_list])
        for i in range(randomnumber-1,randomnumber):
            try:
            #grad_list2 = torch.autograd.grad(grad_i[i], model.parameters(), create_graph=True)
                b = torch.cat([g.view(-1) for g in torch.autograd.grad(grad_i[i], model.parameters(), create_graph=False)])
                a = a.add(b)
                #a = torch.cat((a.cpu(), b.cpu()), 0).cpu()
                del b
            except NameError:
                a = b
    import seaborn as sns
    sns.set(font_scale=1.5)
    datamat = a.cpu().detach().numpy()
    plt.rcParams["figure.figsize"] = (3,3)
    #plt.xticks([float('{:.0e}'.format(0))]+[float('{:.1e}'.format(min(data)))]+[float('{:.1e}'.format(max(data)))])
    plt.yticks([0])
    #plt.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
    sns.distplot(datamat, hist=True, kde=False, 
                 bins=300, color = 'darkblue', 
                 hist_kws={'edgecolor':'black','log':True},
                 kde_kws={'linewidth': 4}
                 )
    plt.savefig('row_matrix_elems_row_'+str(k)+'_'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')
    plt.show()
    sio.savemat('matrix_row_'+str(k)+'_'+dataset+'_'+network+'_'+str(epochsave)+'.mat', {'datamat':datamat})
    counter = 0
    threshold = 1e-6
    for i in range(0,len(a)):
        if abs(a[i]) < threshold:
            counter = counter +1
    print(counter/len(a))
    zerocounts.append(counter/len(a))
    print('time taken = '+str(time.time()-t0))
    del a


0


KeyboardInterrupt: 

In [7]:
print(np.mean(zerocounts))
print(np.std(zerocounts))

0.6897689274293225
0.22767306470419346


In [ ]:
zerocounts = []
sns.set(font_scale=1.5)
#datamat = a.cpu().detach().numpy()
plt.rcParams["figure.figsize"] = (3,3)
plt.xticks[0]
#plt.xticks([float('{:.1e}'.format(0))]+[float('{:.1e}'.format(min(datamat)))]+[float('{:.1e}'.format(max(datamat)))])
#plt.yticks([0])
#plt.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
#plt.ticklabel_format(axis="x,y", style="sci", scilimits=(0,-4),useMathText=True)
sns.distplot(datamat, hist=True, kde=False,
             bins=300, color = 'darkblue', 
             hist_kws={'edgecolor':'black','log':True},
             kde_kws={'linewidth': 4}
             )
plt.ticklabel_format(axis="x", style="sci")
plt.savefig('row_matrix_elems_row_'+str(randomnumber)+'_'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')
plt.show()
sio.savemat('matrix_row_'+str(k)+'_'+dataset+'_'+network+'_'+str(epochsave)+'.mat', {'datamat':datamat})
counter = 0
threshold = 1e-8
for i in range(0,len(a)):
    if abs(a[i]) < threshold:
        counter = counter +1
print(counter/len(a))
zerocounts.append(counter/len(a))

In [ ]:
'{:.1e}'.format(0)

In [ ]:
sns.set(font_scale=1)
#datamat = a.detach().numpy()
plt.rcParams["figure.figsize"] = (3,3)
#plt.xticks([float('{:.0e}'.format(0))]+[float('{:.1e}'.format(min(datamat)))]+[float('{:.1e}'.format(max(datamat)))])
#plt.yticks([0])
#plt.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
sns.distplot(datamat, hist=True, kde=False, 
             bins=300, color = 'darkblue', 
             hist_kws={'edgecolor':'black','log':True},
             kde_kws={'linewidth': 4}
             )
plt.ticklabel_format(style='sci',axis='x')
plt.savefig('row_matrix_elems_row_'+str(randomnumber)+'_'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')
plt.show()

In [ ]:
np.save('row_matrix_elems_row_'+str(randomnumber)+'_'+dataset+'_'+network+'_'+str(epochsave)+'datamat', datamat)

In [ ]:
len(datamat)/num_parametrs

In [ ]:
import scipy.io as sio
sns.distplot(data, hist=True, kde=False, 
             bins=300, color = 'darkblue', 
             hist_kws={'edgecolor':'black','log':True},
             kde_kws={'linewidth': 4}
             )
plt.savefig('row_matrix_elems_row_'+str(randomnumber)+'_'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')
sio.savemat('matrix_row_'+str(randomnumber)+'.mat', {'a':a})

In [ ]:
    counter = 0
    threshold = 1e-8
    for i in range(0,len(a)):
        if abs(a[i]) < threshold:
            counter = counter +1
    print(counter/len(a))

In [ ]:
counter/len(a)

In [ ]:
countern = 0
threshold = 1e-5
for i in range(0,len(a)):
    if abs(a[i]) < threshold:
        counter = counter +1
print(countern/len(a))

In [ ]:
print('max val '+str(torch.max(abs(a))))
print('min val ' +str(torch.min(a)))

In [ ]:
float('{:.2e}'.format(min(data)))

In [ ]:
import seaborn as sns
sns.set(font_scale=1.5)
data = a.detach().numpy()
plt.rcParams["figure.figsize"] = (4,1.5)
plt.xticks([float('{:.0e}'.format(0))]+[float('{:.1e}'.format(min(data)))]+[float('{:.1e}'.format(max(data)))])
plt.yticks([0])
#plt.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
sns.distplot(data, hist=True, kde=False, 
             bins=300, color = 'darkblue', 
             hist_kws={'edgecolor':'black','log':True},
             kde_kws={'linewidth': 4}
             )
plt.savefig('row_matrix_elems'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')


In [ ]:
import matplotlib.ticker as mtick
plt.gca().set_major_formatter(mtick.FormatStrFormatter('%.2e'))

In [ ]:
matrix = a.detach().numpy()
C = np.linalg.svd(matrix, full_matrices=True)
print(C[1][0:30])

In [ ]:
import scipy.io as sio
sio.savemat('matrix_'+dataset+'_'+network+'_'+str(epochsave)+'.mat', {'matrix':matrix})

print('matrix_'+dataset+'_'+network+'_'+str(epochsave)+'.mat saved')

import matplotlib
#ax = sns.heatmap(matrix, yticklabels=2, xticklabels=False)
#uniform_data = np.random.rand(10000, 10000)
plt.imshow(matrix)
cax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(cax=cax)
plt.savefig('matrix_'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')
plt.show()

In [ ]:
#Eigenvalue distribution
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=C[1], bins=10000, color='#0504aa',
                            alpha=0.7, rwidth=0.001)
# plt.grid(axis='y', alpha=0.75)
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.title('My Very Own Histogram')
# plt.text(23, 45, r'$\mu=15, b=3$')
# maxfreq = n.max()
# # Set a clean upper y-axis limit.
# plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)


In [ ]:
counter = 0
threshold = 1e-4
for i in range(0,len(C[1])):
    if C[1][i] < threshold:
        counter = counter+1
print(counter)

In [ ]:
len(matrix[0])

In [ ]:
[max(a) for a in matrix]

In [ ]:
plt.figure(ax)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model.to(device)

In [ ]:
class CurvVecProduct(object):
    def __init__(self, loader, model, criterion, curvature_matrix, full_loader=None):
        self.loader = loader
        self.full_loader = full_loader
        self.model = model
        self.criterion = criterion
        self.iters = 0
        self.timestamp = time.time()
        self.curvature_matrix = curvature_matrix

    def __call__(self, vector):
        start_time = time.time()
        if self.curvature_matrix == 'hessian':
            output = utils.hess_vec(
                vector,
                self.loader,
                self.model,
                self.criterion,
                cuda='cuda'
               
                )
        elif self.curvature_matrix == 'covgrad':
            output = utils.covgrad_vec(
                vector,
                self.loader,
                self.model,
                self.criterion,
                cuda='cuda',
            )
        else:
            raise ValueError("Unrecognised curvature_matrix argument "+self.curvature_matrix)
        time_diff = time.time() - start_time

        self.iters += 1
        print('Iter %d. Time: %.2f' % (self.iters, time_diff))
        # return output.unsqueeze(1)¬
        return output.cpu().unsqueeze(1)

w = torch.cat([param.detach().cpu().view(-1) for param in model.parameters()])
w_l2_norm = torch.norm(w).numpy()
w_linf_norm = torch.norm(w, float('inf')).numpy()

In [ ]:
#Hessian
productor = CurvVecProduct(full_loader, model, criterion, 'hessian')
#utils.bn_update(loader, model)
Q, T = lanczos_tridiag(productor, 15, dtype=torch.float32, device='cpu', matrix_shape=(num_parametrs, num_parametrs))

eigvals, eigvects = T.eig(eigenvectors=True)
gammas = eigvects[0, :] ** 2
V = eigvects.t() @ Q.t()

In [ ]:
eigval = []
for i in range(0,len(eigvals)):
    eigval.append(eigvals[i][0])
mean = np.dot(eigval,gammas)
print(mean)
eigval, gammas

In [ ]:
grad = utils.grad(loader, model, criterion, cuda='cuda' == 'cuda', bn_train_mode=False)

#Grad distribution
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=grad.cpu().numpy(), bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=torch.cat((checkpoint['state_dict']['layer.weight'].view(-1),checkpoint['state_dict']['layer.bias']),0).cpu().numpy(), bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

